In [1]:
import os
from moviepy.editor import VideoFileClip

def split_into_one_minute_videos(input_path, output_folder="one_minute_videos"):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Load the original video
    video = VideoFileClip(input_path)
    duration = int(video.duration)  # total seconds

    chunk_length = 60  # 1 minute

    part = 1
    for start in range(0, duration, chunk_length):
        end = min(start + chunk_length, duration)

        print(f"Creating clip {part}: {start}s to {end}s")

        subclip = video.subclip(start, end)

        output_path = os.path.join(output_folder, f"part_{part:03d}.mp4")

        subclip.write_videofile(
            output_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile=f"temp_audio_{part}.m4a",
            remove_temp=True,
            verbose=False,
            logger=None
        )

        part += 1

    video.close()
    print("Done! All clips created.")


In [2]:
split_into_one_minute_videos("GL010055.mp4", output_folder="one_minute_videos")

Creating clip 1: 0s to 60s
Creating clip 2: 60s to 120s
Creating clip 3: 120s to 180s
Creating clip 4: 180s to 240s
Creating clip 5: 240s to 300s
Creating clip 6: 300s to 360s
Creating clip 7: 360s to 420s
Creating clip 8: 420s to 480s
Creating clip 9: 480s to 540s
Creating clip 10: 540s to 600s
Creating clip 11: 600s to 660s
Creating clip 12: 660s to 720s
Creating clip 13: 720s to 780s
Creating clip 14: 780s to 840s
Creating clip 15: 840s to 900s
Creating clip 16: 900s to 960s
Creating clip 17: 960s to 1020s
Creating clip 18: 1020s to 1080s
Creating clip 19: 1080s to 1140s
Creating clip 20: 1140s to 1200s
Creating clip 21: 1200s to 1260s
Creating clip 22: 1260s to 1320s
Creating clip 23: 1320s to 1380s
Creating clip 24: 1380s to 1440s
Creating clip 25: 1440s to 1500s
Creating clip 26: 1500s to 1560s
Creating clip 27: 1560s to 1620s
Creating clip 28: 1620s to 1680s
Creating clip 29: 1680s to 1740s
Creating clip 30: 1740s to 1800s
Creating clip 31: 1800s to 1860s
Creating clip 32: 1860s

KeyboardInterrupt: 

In [ ]:
from google import genai
import time
import os


client = genai.Client(api_key=API_KEY)

FOLDER = "one_minute_videos"

responses = []   # <- will store all outputs here

# Get all .mp4 clips in the folder
clips = sorted([
    f for f in os.listdir(FOLDER)
    if f.lower().endswith(".mp4")
])

print(f"Found {len(clips)} clips.")


for clip in clips:
    clip_path = os.path.join(FOLDER, clip)
    print(f"\n=== Processing {clip} ===")

    # Upload the clip
    myfile = client.files.upload(file=clip_path)

    # Wait until ready
    while True:
        myfile = client.files.get(name=myfile.name)
        print("State:", myfile.state)

        if myfile.state == "ACTIVE":
            break
        if myfile.state == "FAILED":
            print(f"❌ File failed: {clip}")
            break

        time.sleep(5)

    print("Video ready:", myfile.name)

    prompt = """
    Tell me the timestamp of a team scoring, be sure that the net moves. It might happen that there is no goal, or more than 1. Return **only** the list of timestamps
    of goals, something like this: [00:00:12, 00:00:29, ...], in case there is no goal then [].
    **IMPORTANT**: don't output any other character apart from the list
    """

    # Run the Gemini analysis with retry (3 attempts)
    response = None
    for attempt in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash", contents=[myfile, prompt]
            )
            break  # success → exit retry loop
        except Exception as e:
            print(f"⚠️ LLM call failed (attempt {attempt+1}/3): {e}")
            if attempt == 2:  # last attempt
                print("❌ LLM failed after 3 attempts")
            else:
                time.sleep(2)

    if response is None:
        output_text = "[]"
    else:
        output_text = response.text.strip()

    print("Model response:", output_text)

    # Store result in list
    responses.append({
        "clip": clip,
        "file_id": myfile.name,
        "response": output_text
    })


print("\n=== ALL RESPONSES ===")
for r in responses:
    print(f"\nClip: {r['clip']}")
    print(f"File ID: {r['file_id']}")
    print("Response:", r["response"])


Found 33 clips.

=== Processing part_001.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/jnilf859k8ez
Model response: []

=== Processing part_002.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/ldh272aro5e0
Model response: []

=== Processing part_003.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/vic4hgza9bh0
Model response: [00:00:29]

=== Processing part_004.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/a5wa82j3mril
Model response: [00:27]

=== Processing part_005.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/anfpb0wxntqf
Model response: [00:00:53]

=== Processing part_006.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/bh2oybnj995z
Model response: [00:00:20, 00:00:41]

=== Processing part_007.mp4 ===
State: FileState.PROCESSING
State: FileState.ACTIVE
Video ready: files/wwxapc3lqexr
Model respon

In [ ]:
responses

[{'clip': 'part_001.mp4',
  'file_id': 'files/nj8eq328fcbj',
  'response': '[00:00:52]'},
 {'clip': 'part_002.mp4',
  'file_id': 'files/0vn0k0ihlwjb',
  'response': '[00:00:21]'},
 {'clip': 'part_003.mp4',
  'file_id': 'files/u8954kvpyk3g',
  'response': '[00:00:29]'},
 {'clip': 'part_004.mp4', 'file_id': 'files/5je4jgdvn4mf', 'response': '[]'},
 {'clip': 'part_005.mp4',
  'file_id': 'files/2wl9yts6gg7q',
  'response': '[00:00:16, 00:00:21]'}]

In [4]:
import re

def time_to_seconds(t):
    h, m, s = map(int, t.split(":"))
    return h*3600 + m*60 + s

def seconds_to_time(sec):
    h = sec // 3600
    m = (sec % 3600) // 60
    s = sec % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

processed_times = []  # final list

for i, item in enumerate(responses):
    clip_offset = i * 60   # seconds added per clip
    
    # Extract timestamps from the response
    timestamps = re.findall(r"\d{2}:\d{2}:\d{2}", item["response"])
    
    for t in timestamps:
        absolute_sec = time_to_seconds(t) + clip_offset
        absolute_time = seconds_to_time(absolute_sec)
        processed_times.append(absolute_time)

print(processed_times)


['00:02:29', '00:04:53', '00:05:20', '00:05:41', '00:06:29', '00:09:10', '00:10:17', '00:10:32', '00:11:28', '00:12:26', '00:13:08', '00:13:23', '00:13:43', '00:15:13', '00:15:59', '00:16:20', '00:16:31', '00:17:29', '00:19:51', '00:20:48', '00:21:27', '00:22:34', '00:24:37', '00:25:03', '00:26:08', '00:26:29', '00:28:07', '00:29:53', '00:30:50', '00:32:28']


In [5]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def extract_highlights_merged(
    video_path,
    timestamps,        # list of absolute times as "HH:MM:SS"
    pre_seconds=10,
    post_seconds=10,
    output_path="highlights.mp4"
):
    def t_to_sec(t):
        h, m, s = map(int, t.split(":"))
        return h*3600 + m*60 + s

    # Step 1 — Convert timestamps to raw intervals
    raw_intervals = []
    for t in timestamps:
        center = t_to_sec(t)
        raw_intervals.append((center - pre_seconds, center + post_seconds))

    # Step 2 — Sort intervals
    raw_intervals.sort()

    # Step 3 — Merge overlapping intervals
    merged = []
    for start, end in raw_intervals:
        if not merged:
            merged.append([start, end])
        else:
            last_start, last_end = merged[-1]
            if start <= last_end:  
                # overlap → merge
                merged[-1][1] = max(last_end, end)
            else:
                merged.append([start, end])

    print("Merged intervals:", merged)

    # Step 4 — Cut the merged intervals from the video
    video = VideoFileClip(video_path)
    final_clips = []

    for start, end in merged:
        start = max(0, start)
        end = min(video.duration, end)

        print(f"Extracting merged clip: {start} → {end}")
        final_clips.append(video.subclip(start, end))

    # Step 5 — Concatenate into final video
    final = concatenate_videoclips(final_clips)
    final.write_videofile(output_path, codec="libx264", audio_codec="aac")

    print("Done! Saved:", output_path)


In [6]:
extract_highlights_merged(
    "GL010055.mp4",
    processed_times,        # list of absolute times as "HH:MM:SS"
    pre_seconds=5,
    post_seconds=5,
    output_path="highlights_primo_tempo.mp4"
)

Merged intervals: [[144, 154], [288, 298], [315, 325], [336, 346], [384, 394], [545, 555], [612, 622], [627, 637], [683, 693], [741, 751], [783, 793], [798, 808], [818, 828], [908, 918], [954, 964], [975, 985], [986, 996], [1044, 1054], [1186, 1196], [1243, 1253], [1282, 1292], [1349, 1359], [1472, 1482], [1498, 1508], [1563, 1573], [1584, 1594], [1682, 1692], [1788, 1798], [1845, 1855], [1943, 1953]]
Extracting merged clip: 144 → 154
Extracting merged clip: 288 → 298
Extracting merged clip: 315 → 325
Extracting merged clip: 336 → 346
Extracting merged clip: 384 → 394
Extracting merged clip: 545 → 555
Extracting merged clip: 612 → 622
Extracting merged clip: 627 → 637
Extracting merged clip: 683 → 693
Extracting merged clip: 741 → 751
Extracting merged clip: 783 → 793
Extracting merged clip: 798 → 808
Extracting merged clip: 818 → 828
Extracting merged clip: 908 → 918
Extracting merged clip: 954 → 964
Extracting merged clip: 975 → 985
Extracting merged clip: 986 → 996
Extracting merged

MoviePy - Done.
Moviepy - Writing video highlights_primo_tempo.mp4



Moviepy - Done !
Moviepy - video ready highlights_primo_tempo.mp4
Done! Saved: highlights_primo_tempo.mp4
